# Model Validation


In [21]:
import importlib
import pandas as pd
import data_preprocessor as dpp
importlib.reload(dpp)

melbourne_data = dpp.process(pd.read_csv('melb_data.csv'))
melbourne_data.describe()


,rooms,price,distance,postcode,bedroom2,bathroom,car,landsize,buildingarea,yearbuilt,lattitude,longtitude,propertycount
count,6196.000000,6.196000e+03,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.068828e+06,9.751097,3101.947708,2.902034,1.576340,1.573596,471.006940,141.568645,1964.081988,-37.807904,144.990201,7435.489509
std,0.971079,6.751564e+05,5.612065,86.421604,0.970055,0.711362,0.929947,897.449881,90.834824,38.105673,0.075850,0.099165,4337.698917
min,1.000000,1.310000e+05,0.000000,3000.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1196.000000,-38.164920,144.542370,389.000000
25%,2.000000,6.200000e+05,5.900000,3044.000000,2.000000,1.000000,1.000000,152.000000,91.000000,1940.000000,-37.855438,144.926198,4383.750000
50%,3.000000,8.800000e+05,9.000000,3081.000000,3.000000,1.000000,1.000000,373.000000,124.000000,1970.000000,-37.802250,144.995800,6567.000000
75%,4.000000,1.325000e+06,12.400000,3147.000000,3.000000,2.000000,2.000000,628.000000,170.000000,2000.000000,-37.758200,145.052700,10175.000000
max,8.000000,9.000000e+06,47.400000,3977.000000,9.000000,8.000000,10.000000,37000.000000,3112.000000,2018.000000,-37.457090,145.526350,21650.000000


Let's confirm there are now NaN values...

In [22]:
melbourne_data.columns

Index(['suburb', 'address', 'rooms', 'type', 'price', 'method', 'sellerg',
       'date', 'distance', 'postcode', 'bedroom2', 'bathroom', 'car',
       'landsize', 'buildingarea', 'yearbuilt', 'councilarea', 'lattitude',
       'longtitude', 'regionname', 'propertycount'],
      dtype='object')

In [23]:

for col in melbourne_data.columns:
    print(f'{col} has NaN') if melbourne_data[col].hasnans  else None

## Selecting the Prediction Target

In [24]:
model_features = ['rooms', 'bathroom', 'landsize', 'lattitude', 'buildingarea', 'yearbuilt', 'lattitude', 'longtitude']

y = melbourne_data.price
X = melbourne_data[model_features]

X.describe()

,rooms,bathroom,landsize,lattitude,buildingarea,yearbuilt,lattitude,longtitude
count,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000,6196.000000
mean,2.931407,1.576340,471.006940,-37.807904,141.568645,1964.081988,-37.807904,144.990201
std,0.971079,0.711362,897.449881,0.075850,90.834824,38.105673,0.075850,0.099165
min,1.000000,1.000000,0.000000,-38.164920,0.000000,1196.000000,-38.164920,144.542370
25%,2.000000,1.000000,152.000000,-37.855438,91.000000,1940.000000,-37.855438,144.926198
50%,3.000000,1.000000,373.000000,-37.802250,124.000000,1970.000000,-37.802250,144.995800
75%,4.000000,2.000000,628.000000,-37.758200,170.000000,2000.000000,-37.758200,145.052700
max,8.000000,8.000000,37000.000000,-37.457090,3112.000000,2018.000000,-37.457090,145.526350


In [25]:
X.head()

,rooms,bathroom,landsize,lattitude,buildingarea,yearbuilt,lattitude,longtitude
1,2,1.0,156.0,-37.8079,79.0,1900.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,150.0,1900.0,-37.8093,144.9944
4,4,1.0,120.0,-37.8072,142.0,2014.0,-37.8072,144.9941
6,3,2.0,245.0,-37.8024,210.0,1910.0,-37.8024,144.9993
7,2,1.0,256.0,-37.8060,107.0,1890.0,-37.8060,144.9954


## Training with Out-of-Sample Data
We should not test our model with the data we used to train it.  We can split the data into two pieces, so the model is trained and tested on different data.  This is called **splitting** the data.

In [27]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    return mean_absolute_error(val_y, preds_val)

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=0)
melbourne_model = DecisionTreeRegressor(random_state=1)
melbourne_model.fit(train_X, train_y)

for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print(f'Max leaf nodes: {max_leaf_nodes} \t\t Mean Absolute Error: {my_mae}')

Max leaf nodes: 5 		 Mean Absolute Error: 347380.33833344496
Max leaf nodes: 50 		 Mean Absolute Error: 258171.21202406782
Max leaf nodes: 500 		 Mean Absolute Error: 249434.099896096
Max leaf nodes: 5000 		 Mean Absolute Error: 261836.22401549388


The out of sample mean absolute error (MAE) is the average error we see when we use our model on data that it was not trained on.  This tells us how well our model is likely to do when used in the real world. In this case, we performed horribly and we need to improve the model.